In [1]:
import cv2
import numpy as np

# Функция для создания маски из изображения с желтым контуром
def create_mask_from_image(image_path, output_path, min_contour_area=2500):
    # Загрузим изображение
    image = cv2.imread(image_path)

    # Преобразуем изображение в HSV (для более точного определения желтого цвета)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Диапазон желтого цвета в формате HSV
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Найдем все пиксели, которые попадают в диапазон желтого цвета
    yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)

    # Морфологическая операция - расширение (dilation), чтобы соединить пунктирные линии
    kernel = np.ones((5, 5), np.uint8)
    yellow_mask_dilated = cv2.dilate(yellow_mask, kernel, iterations=2)

    # Дополнительно применим операцию закрытия (closing), чтобы лучше соединить разрывы
    yellow_mask_closed = cv2.morphologyEx(yellow_mask_dilated, cv2.MORPH_CLOSE, kernel)

    # Найдем контуры на основе обработанной маски
    contours, _ = cv2.findContours(yellow_mask_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Создадим пустую маску (черную)
    mask = np.zeros(image.shape[:2], dtype=np.uint8)

    # Фильтруем и рисуем только большие контуры (площадь > min_contour_area)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_contour_area:
            # Заполняем контур, если его площадь больше заданного порога
            cv2.drawContours(mask, [contour], -1, (255), thickness=cv2.FILLED)

    # Сохраним полученную бинарную маску как PNG
    cv2.imwrite(output_path, mask)

    print(f'Маска успешно сохранена в {output_path}')

In [8]:
image_path = 'data\\sech_2\\2024.03.28_2\\Image58.jpg'  # Путь к вашему изображению с желтым контуром
output_path = 'data\\sech_2\\2024.03.28_2\\Image58_mask.png'  # Путь для сохранения маски
create_mask_from_image(image_path, output_path)

Маска успешно сохранена в data\sech_2\2024.03.28_2\Image58_mask.png


In [7]:
import cv2
import numpy as np

# Шаг 1: Загрузка изображения
image = cv2.imread('data\\sech_2\\2024.03.28_1\\Image47_m.jpg')

# Шаг 2: Преобразование в цветовое пространство HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Шаг 3: Определение диапазона зелёного цвета в HSV
lower_green = np.array([40, 40, 40])  # Нижняя граница зелёного (настраивается)
upper_green = np.array([80, 255, 255])  # Верхняя граница зелёного (настраивается)

# Поиск маски зелёных областей
mask = cv2.inRange(hsv, lower_green, upper_green)

# Шаг 4: Нахождение контуров на основе маски
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Шаг 5: Создание пустого чёрно-белого изображения для маски
binary_mask = np.zeros_like(mask)

# Заливка контуров белым цветом
cv2.drawContours(binary_mask, contours, -1, (255), thickness=cv2.FILLED)

# Шаг 6: Сохранение результата как PNG
cv2.imwrite('data\\sech_2\\2024.03.28_1\\Image47_mask.png', binary_mask)


True

In [22]:
import os
import glob
import cv2

def find_masked_png_files(directory, kol):
    im = 'data\\YOLO_sech_1\\images\\'
    labl = 'data\\YOLO_sech_1\\labels\\'
    # Проходим по всем элементам в заданной папке
    for root, dirs, files in os.walk(directory):
        # Ищем папки с окончанием "-masked"
        for dir_name in dirs:
            if dir_name.endswith('-masked'):
                masked_dir_path = os.path.join(root, dir_name)
                # Ищем все файлы с расширением .png внутри папки
                png_files = glob.glob(os.path.join(masked_dir_path, '*.png'))
                
                if png_files:
                    print(f"Найдены PNG файлы в папке: {masked_dir_path}")
                    for png_file in png_files:
                        fin_p1 = png_file[:-4] + '_fin' + str(kol) +'.png'
                        fin_p2 = im + str(kol) +'.jpg'
                        txt_p1 = png_file[:-4] + '_fin' + str(kol) +'.txt'
                        txt_p2 = labl + str(kol) +'.txt'
                        im_p = png_file.split('_mask')[0] + '.jpg'
                        process_mask(png_file, fin_p1)
                        imag = cv2.imread(im_p)
                        cv2.imwrite(fin_p2, imag)
                        yolo_bboxes = get_yolo_bounding_boxes(fin_p1, class_id=0)  # class_id=0 можно изменить на нужный класс
                        save_yolo_bboxes_to_txt(yolo_bboxes, txt_p1)
                        save_yolo_bboxes_to_txt(yolo_bboxes, txt_p2)
                        kol += 1

# Пример использования функции
if __name__ == "__main__":
    kol = 0
    # Укажите путь к основной папке, которую хотите просканировать
    directory_to_search = "data\\sech_1\\Sechenov_dataset_revision_v01"
    find_masked_png_files(directory_to_search, kol)


Найдены PNG файлы в папке: data\sech_1\Sechenov_dataset_revision_v01\2023.05.03-masked
Обработанное изображение сохранено по пути: data\sech_1\Sechenov_dataset_revision_v01\2023.05.03-masked\img1_mask_fin0.png
Найдены PNG файлы в папке: data\sech_1\Sechenov_dataset_revision_v01\2023.05.31-masked
Обработанное изображение сохранено по пути: data\sech_1\Sechenov_dataset_revision_v01\2023.05.31-masked\img1_mask2_fin1.png
Обработанное изображение сохранено по пути: data\sech_1\Sechenov_dataset_revision_v01\2023.05.31-masked\img2_mask2_fin2.png
Найдены PNG файлы в папке: data\sech_1\Sechenov_dataset_revision_v01\2023.06.01-masked
Обработанное изображение сохранено по пути: data\sech_1\Sechenov_dataset_revision_v01\2023.06.01-masked\img1_mask2_fin3.png
Обработанное изображение сохранено по пути: data\sech_1\Sechenov_dataset_revision_v01\2023.06.01-masked\img2_mask2_fin4.png
Найдены PNG файлы в папке: data\sech_1\Sechenov_dataset_revision_v01\2023.06.08-masked
Обработанное изображение сохранен

Обработанное изображение сохранено по пути: data\sech_1\Sechenov_dataset_revision_v01\2023.11.03-masked\img2_mask2_fin44.png
Обработанное изображение сохранено по пути: data\sech_1\Sechenov_dataset_revision_v01\2023.11.03-masked\img3_mask2_fin45.png
Обработанное изображение сохранено по пути: data\sech_1\Sechenov_dataset_revision_v01\2023.11.03-masked\img4_mask2_fin46.png
Найдены PNG файлы в папке: data\sech_1\Sechenov_dataset_revision_v01\2023.11.03_1-masked
Обработанное изображение сохранено по пути: data\sech_1\Sechenov_dataset_revision_v01\2023.11.03_1-masked\img1_mask2_fin47.png
Найдены PNG файлы в папке: data\sech_1\Sechenov_dataset_revision_v01\2023.11.20-masked
Обработанное изображение сохранено по пути: data\sech_1\Sechenov_dataset_revision_v01\2023.11.20-masked\img1_mask2_fin48.png
Обработанное изображение сохранено по пути: data\sech_1\Sechenov_dataset_revision_v01\2023.11.20-masked\img2_mask2_fin49.png
Обработанное изображение сохранено по пути: data\sech_1\Sechenov_dataset

In [12]:
from PIL import Image

def process_mask(image_path, save_path):
    # Открываем изображение
    img = Image.open(image_path)

    # Преобразуем изображение в RGB, если оно не в RGB формате
    img = img.convert("RGB")

    # Загружаем пиксели изображения
    pixels = img.load()

    # Проходим по каждому пикселю изображения
    for i in range(img.width):
        for j in range(img.height):
            r, g, b = pixels[i, j]
            
            brightness = (r * 0.299 + g * 0.587 + b * 0.114)

            # Если яркость выше порога, делаем пиксель белым, иначе черным
            if brightness > threshold:
                pixels[i, j] = (255, 255, 255)  # Белый
            else:
                pixels[i, j] = (0, 0, 0)  # Черный

    # Сохраняем результат
    img.save(save_path)
    print(f"Обработанное изображение сохранено по пути: {save_path}")

In [16]:
from PIL import Image
import numpy as np
from scipy.ndimage import label

def get_yolo_bounding_boxes(image_path, class_id=0):
    # Открываем изображение и конвертируем в RGB
    img = Image.open(image_path).convert("RGB")
    
    # Преобразуем изображение в numpy массив
    img_np = np.array(img)
    
    # Превращаем изображение в бинарную маску (если пиксель не черный, то считаем его белым)
    mask = np.any(img_np != [0, 0, 0], axis=-1).astype(np.int32)
    
    # Находим связанные компоненты (отдельные объекты) на маске
    labeled_mask, num_features = label(mask)
    
    # Получаем размеры изображения
    img_height, img_width = img_np.shape[:2]
    
    yolo_bboxes = []
    
    # Для каждого найденного объекта находим bounding box
    for label_id in range(1, num_features + 1):
        # Находим индексы, где значение метки равно текущему объекту
        indices = np.where(labeled_mask == label_id)
        
        min_y, min_x = np.min(indices[0]), np.min(indices[1])
        max_y, max_x = np.max(indices[0]), np.max(indices[1])
        
        # Вычисляем ширину, высоту и центр bounding box в пикселях
        bbox_width = max_x - min_x
        bbox_height = max_y - min_y
        x_center = min_x + bbox_width / 2
        y_center = min_y + bbox_height / 2
        
        # Нормализуем значения в диапазон [0, 1]
        x_center_norm = x_center / img_width
        y_center_norm = y_center / img_height
        bbox_width_norm = bbox_width / img_width
        bbox_height_norm = bbox_height / img_height
        
        # Добавляем bounding box в YOLO формате
        yolo_bboxes.append((class_id, x_center_norm, y_center_norm, bbox_width_norm, bbox_height_norm))
    
    return yolo_bboxes

def save_yolo_bboxes_to_txt(yolo_bboxes, output_txt_path):
    # Открываем файл для записи
    with open(output_txt_path, 'w') as f:
        for bbox in yolo_bboxes:
            # YOLO формат: <class_id> <x_center_norm> <y_center_norm> <bbox_width_norm> <bbox_height_norm>
            class_id, x_center_norm, y_center_norm, bbox_width_norm, bbox_height_norm = bbox
            # Записываем bounding box в строку в формате YOLO
            f.write(f"{class_id} {x_center_norm:.6f} {y_center_norm:.6f} {bbox_width_norm:.6f} {bbox_height_norm:.6f}\n")


## train val split

In [1]:
import os
import random
import shutil

# Параметры
images_path = "data\\YOLO_sech_1\\images"  # Путь к папке с изображениями
annotations_path = "data\\YOLO_sech_1\\labels"  # Путь к папке с аннотациями
output_path = "data\\YOLO_sech_1"  # Путь для сохранения разбитого датасета
train_ratio = 0.8  # Доля данных для тренировки (оставшееся пойдет на валидацию)

# Создаем папки для train и val
train_images_path = os.path.join(output_path, "train", "images")
train_labels_path = os.path.join(output_path, "train", "labels")
val_images_path = os.path.join(output_path, "val", "images")
val_labels_path = os.path.join(output_path, "val", "labels")

os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)
os.makedirs(val_labels_path, exist_ok=True)

# Собираем все изображения и аннотации
images = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png'))]
random.shuffle(images)  # Перемешиваем данные

# Разделение на train и val
train_count = int(len(images) * train_ratio)
train_images = images[:train_count]
val_images = images[train_count:]

# Копируем файлы в соответствующие папки
def move_files(image_list, src_images_path, src_annotations_path, dest_images_path, dest_labels_path):
    for image_name in image_list:
        # Путь к изображению и аннотации
        image_path = os.path.join(src_images_path, image_name)
        label_name = image_name.replace('.jpg', '.txt').replace('.png', '.txt')
        label_path = os.path.join(src_annotations_path, label_name)

        # Копируем изображение
        shutil.copy(image_path, os.path.join(dest_images_path, image_name))
        # Копируем аннотацию (если она существует)
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(dest_labels_path, label_name))

# Переносим файлы
move_files(train_images, images_path, annotations_path, train_images_path, train_labels_path)
move_files(val_images, images_path, annotations_path, val_images_path, val_labels_path)

print("Разделение завершено!")


Разделение завершено!
